In [39]:
import random

import numpy as np
import pandas as pd
import itertools

In [40]:
df_initial = pd.read_csv("data/source/train_data.csv")

In [41]:
df_initial.head()

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,1565607600,30.329370,59.940488,63,4,52,6,0,0,0,0
1,1565568000,30.342845,59.869542,44,0,40,0,4,0,0,0
2,1565568000,30.322632,60.039332,30,0,77,7,0,0,0,1
3,1565568000,30.322632,59.938238,13,0,634,17,32,0,0,7
4,1565568000,30.309158,59.932612,8,0,0,0,0,0,0,0


In [42]:
# translate time to hours
df_initial["timestamp"] /= 60 * 60
df_initial["timestamp"] = df_initial["timestamp"].apply(int)

In [43]:
df_initial.head()

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,434891,30.329370,59.940488,63,4,52,6,0,0,0,0
1,434880,30.342845,59.869542,44,0,40,0,4,0,0,0
2,434880,30.322632,60.039332,30,0,77,7,0,0,0,1
3,434880,30.322632,59.938238,13,0,634,17,32,0,0,7
4,434880,30.309158,59.932612,8,0,0,0,0,0,0,0


In [44]:
df_initial.shape

(8526526, 11)

In [45]:
size = 0.25 # wild guess
spb_lon, spb_lat = 30.3141, 59.9386 # Spb coords
df = df_initial[(df_initial["lon"] <= spb_lon + size / 2.0) & (df_initial["lon"] >= spb_lon - size / 2.0) &
                (df_initial["lat"] <= spb_lat + size / 2.0) & (df_initial["lat"] >= spb_lat - size / 2.0)]

In [46]:
df["count"] = 1
df = df.groupby(["lon", "lat", "timestamp"]).agg({
    "count": np.sum,
    "likescount": np.mean,
    "commentscount": np.mean,
    "symbols_cnt": np.mean,
    "words_cnt": np.mean,
    "hashtags_cnt": np.mean,
    "mentions_cnt": np.mean,
    "links_cnt": np.mean,
    "emoji_cnt": np.mean,
}).reset_index()

/tmp/ipykernel_175794/1201879296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["count"] = 1


In [47]:
df.head()

,lon,lat,timestamp,count,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,30.190131,59.848116,431127,1,30.0,2.0,36.0,3.0,0.0,0.0,0.0,3.0
1,30.190131,59.848116,431422,1,7.0,0.0,256.0,0.0,12.0,0.0,0.0,0.0
2,30.190131,59.848116,431491,1,22.0,0.0,522.0,0.0,28.0,0.0,0.0,0.0
3,30.190131,59.848116,431955,1,16.0,3.0,60.0,0.0,4.0,0.0,0.0,0.0
4,30.190131,59.848116,432821,1,30.0,1.0,618.0,36.0,30.0,0.0,0.0,0.0


In [48]:
df.head()

,lon,lat,timestamp,count,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,30.190131,59.848116,431127,1,30.0,2.0,36.0,3.0,0.0,0.0,0.0,3.0
1,30.190131,59.848116,431422,1,7.0,0.0,256.0,0.0,12.0,0.0,0.0,0.0
2,30.190131,59.848116,431491,1,22.0,0.0,522.0,0.0,28.0,0.0,0.0,0.0
3,30.190131,59.848116,431955,1,16.0,3.0,60.0,0.0,4.0,0.0,0.0,0.0
4,30.190131,59.848116,432821,1,30.0,1.0,618.0,36.0,30.0,0.0,0.0,0.0


In [49]:
unique_lon = df.lon.unique()
unique_lat = df.lat.unique()

In [50]:
len(unique_lon), len(unique_lat)

(111, 196)

In [51]:
cartesian_prod_lon_lat = list(map(list, zip(*itertools.product(unique_lon, unique_lat))))

In [52]:
df_full = pd.DataFrame({"lon": cartesian_prod_lon_lat[0], "lat":cartesian_prod_lon_lat[1]})
df_full.head()

,lon,lat
0,30.190131,59.848116
1,30.190131,59.850372
2,30.190131,59.851500
3,30.190131,59.852628
4,30.190131,59.853756


In [53]:
df_full = pd.merge(df_full, df, on=["lon", "lat"], how="outer")
df_full.head()

,lon,lat,timestamp,count,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,30.190131,59.848116,431127.0,1.0,30.0,2.0,36.0,3.0,0.0,0.0,0.0,3.0
1,30.190131,59.848116,431422.0,1.0,7.0,0.0,256.0,0.0,12.0,0.0,0.0,0.0
2,30.190131,59.848116,431491.0,1.0,22.0,0.0,522.0,0.0,28.0,0.0,0.0,0.0
3,30.190131,59.848116,431955.0,1.0,16.0,3.0,60.0,0.0,4.0,0.0,0.0,0.0
4,30.190131,59.848116,432821.0,1.0,30.0,1.0,618.0,36.0,30.0,0.0,0.0,0.0


In [54]:
df_full.fillna(0, inplace=True)

In [55]:
df_full[df_full.columns[~df_full.columns.isin(["lon","lat"])]] = df_full[df_full.columns[~df_full.columns.isin(["lon","lat"])]].astype("int64")

In [56]:
df_full["hour"] = df_full.timestamp % 24

In [57]:
df_with_features = pd.get_dummies(df_full, prefix="hour", columns=["hour"])
df_with_features.head()

,lon,lat,timestamp,count,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,30.190131,59.848116,431127,1,30,2,36,3,0,0,...,0,1,0,0,0,0,0,0,0,0
1,30.190131,59.848116,431422,1,7,0,256,0,12,0,...,0,0,0,0,0,0,0,0,1,0
2,30.190131,59.848116,431491,1,22,0,522,0,28,0,...,0,0,0,0,0,1,0,0,0,0
3,30.190131,59.848116,431955,1,16,3,60,0,4,0,...,0,0,0,0,0,0,0,0,0,0
4,30.190131,59.848116,432821,1,30,1,618,36,30,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
df_with_features.to_csv("./data/processed_data.csv", index=False)